<h1>Translate description from other language to english</h1>

In [7]:
import pandas as pd
import time
from deep_translator import GoogleTranslator
from langdetect import detect, LangDetectException
from multiprocessing import Pool, cpu_count
from tqdm import tqdm  # Progress bar library

# CSV Import
BaseDataFrame = pd.read_csv("new_data/books_corrected.csv", delimiter=",")

# Define a function to handle translation with retry logic
def translate_with_retry(text, retries=3, delay=2):
    for attempt in range(retries):
        try:
            return GoogleTranslator(source='auto', target='en').translate(text)
        except Exception as e:
            if attempt < retries - 1:
                time.sleep(delay)
            else:
                return text  # Return original text on failure

# Function to check if text has enough features for language detection
def has_text_features(text):
    return any(char.isalpha() for char in text)

# Function to process a single row
def process_row(row):
    description = row.get("description", None)

    if isinstance(description, str) and description.strip():  # Check if description is a non-empty string
        if has_text_features(description):
            try:
                if detect(description) != 'en':
                    # Attempt translation with retries
                    row["description"] = translate_with_retry(description)
            except LangDetectException:
                pass  # Skip rows where language detection fails
    return row

# Parallel processing using multiprocessing
if __name__ == "__main__":
    with Pool(cpu_count()) as pool:
        updated_rows = list(tqdm(pool.imap(process_row, [row for _, row in BaseDataFrame.iterrows()]), total=len(BaseDataFrame), desc="Translating rows"))

    # Convert the updated rows back to a DataFrame
    updated_df = pd.DataFrame(updated_rows)

    # Save the updated DataFrame to a new CSV file
    updated_df.to_csv("new_data/BooksCorrectedTranslated.csv", index=False)

    print("Translation and CSV update completed.")

/tmp/ipykernel_4392/3962711216.py:9: DtypeWarning: Columns (18,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86) have mixed types. Specify dtype option on import or set low_memory=False.
  BaseDataFrame = pd.read_csv("new_data/books_corrected.csv", delimiter=",")
Translating rows:   3%|▎         | 1514/52199 [00:07<04:05, 206.66it/s]


KeyboardInterrupt: 

In [8]:
import pandas as pd
from sqlalchemy import create_engine, MetaData, Table, bindparam, update
import configparser

# Step 1: Load Configuration
config = configparser.ConfigParser()
config.read('configure.ini')
print(config.sections())
dbaddress = config['Database']['DB_URL']

# Step 2: Create SQLAlchemy Engine
engine = create_engine(dbaddress)

# Reflect the table metadata
metadata = MetaData()
metadata.reflect(bind=engine)
books_table = metadata.tables['book']  # Replace 'book' with your table name

# Step 3: Load CSV Data
csv_file = "new_data/BooksCorrectedTranslated.csv"  # Path to the CSV
df = pd.read_csv(csv_file)

# Step 4: Prepare Bulk Data for Update
bulk_data = [
    {"csv_book_id": row['id'], "book_description": row['description']}
    for _, row in df.iterrows()
]

# Step 5: Perform Bulk Update
with engine.connect() as connection:
    with connection.begin():  # Start a transaction
        connection.execute(
            books_table.update()
            .where(books_table.c.book_id == bindparam('csv_book_id'))
            .values(book_description=bindparam('book_description')),
            bulk_data
        )

print("Database update completed successfully.")


['Database']


OperationalError: (psycopg2.OperationalError) SSL connection has been closed unexpectedly

(Background on this error at: https://sqlalche.me/e/20/e3q8)

In [1]:
import pandas as pd
from sqlalchemy import create_engine, MetaData, Table, bindparam, update
import configparser
from tqdm import tqdm  # Import tqdm for progress bar

# Step 1: Load Configuration
config = configparser.ConfigParser()
config.read('configure.ini')
print(config.sections())
dbaddress = config['Database']['DB_URL']

# Step 2: Create SQLAlchemy Engine
engine = create_engine(dbaddress)

# Reflect the table metadata
metadata = MetaData()
metadata.reflect(bind=engine)
books_table = metadata.tables['book']  # Replace 'book' with your table name

# Step 3: Load CSV Data
csv_file = "new_data/BooksCorrectedTranslated.csv"  # Path to the CSV
df = pd.read_csv(csv_file)

# Step 4: Prepare Bulk Data for Update
bulk_data = [
    {"csv_book_id": row['id'], "book_description": row['description']}
    for _, row in tqdm(df.iterrows(), total=len(df), desc="Preparing data")
]

# Step 5: Perform Bulk Update with Progress Bar
with engine.connect() as connection:
    with connection.begin():  # Start a transaction
        for batch in tqdm(bulk_data, total=len(bulk_data), desc="Updating database"):
            connection.execute(
                books_table.update()
                .where(books_table.c.book_id == bindparam('csv_book_id'))
                .values(book_description=bindparam('book_description')),
                [batch]  # Execute one update at a time with batching
            )

print("Database update completed successfully.")


/usr/lib/python3/dist-packages/pytz/__init__.py:31: SyntaxWarning: invalid escape sequence '\s'
  match = re.match("^#\s*version\s*([0-9a-z]*)\s*$", line)


['Database']


/tmp/ipykernel_3825/1576254144.py:22: DtypeWarning: Columns (18,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_file)
Updating database: 100%|██████████| 52199/52199 [24:10<00:00, 35.99it/s]  


Database update completed successfully.


In [5]:
import pandas as pd
from sqlalchemy import create_engine, select, Table, MetaData, update
from sqlalchemy.orm import sessionmaker
import yaml
import time
from tqdm import tqdm

# Import la donnée du fichier yml
with open('sprint2_populating/application/config.yml', 'r') as file:
    config = yaml.safe_load(file)


def conexion_db():
    """
    Establishes a connection to the SQL database.
    
    Returns:
        tuple: (engine, session) where:
            - engine: The SQLAlchemy Engine object connected to the database.
            - session: A SQLAlchemy session for executing queries.
    
    Raises:
        Exception: If the connection to the database fails.
    """
    try:
        ## URL of the database
        database_url = config['adress_sql']
        engine = create_engine(database_url)
        session = sessionmaker(bind=engine)
        session = session()
        print("Connection to the database successful")
        return engine, session
    except:
        raise Exception("Error in the connection to the database")

def insert(dataframe, table_name):
    """
    Inserts data from a pandas DataFrame into a SQL table.
    
    Args:
        dataframe (pd.DataFrame): The DataFrame containing data to insert.
        table_name (str): The name of the target table in the database.
    
    Returns:
        bool: True if the data is inserted successfully, False otherwise.
    
    Raises:
        Exception: If connection fails after multiple attempts.
    """
    attempts = 3
    interval = 3
    for attempt in range(attempts):
        try:
            engine = conexion_db()[0]
            break
        except Exception as e:
            if attempt < attempts - 1:
                time.sleep(interval)
            else:
                raise Exception("Failed to connect to the database after multiple attempts") from e
    try:
        dataframe.to_sql(table_name, con=engine, if_exists='append', index=False)
        print("Data inserted into the database")
        return True
    except Exception as e:
        print(f"Error inserting data into the database : {e}")
        return False
    
def insert_table_assocation(dataframe, table1, table2, table1_key, table2_key, table1_id, table2_id, table_name=None):
    """
    Creates and inserts associations between two SQL tables.
    
    Args:
        dataframe (pd.DataFrame): The DataFrame containing data to create associations.
        table1 (str): Name of the first table in the database.
        table2 (str): Name of the second table in the database.
        table1_key (str): Name of the secondary key used to identify records in `table1`.
        table2_key (str): Name of the secondary key used to identify records in `table2`.
        table1_id (str): Name of the column containing the ID in `table1`.
        table2_id (str): Name of the column containing the ID in `table2`.
        table_name (str) (optionnal): Name given to the created associations table.
    
    Returns:
        bool: True if the associations are successfully inserted.
    
    Raises:
        Exception: If any step in the process fails (connection, data retrieval, etc.).
    """
    attempts = 3
    interval = 3
    for attempt in range(attempts):
        try:
            engine = conexion_db()[0]
            session = conexion_db()[1]
            break
        except Exception as e:
            if attempt < attempts - 1:
                time.sleep(interval)
            else:
                raise Exception("Failed to connect to the database after multiple attempts") from e

    try:
        # Load the tables with the engine
        table1 = Table(table1, MetaData(), autoload_with=engine)
        table2 = Table(table2, MetaData(), autoload_with=engine)
    except Exception as e:
        raise Exception("Error loading tables with the engine") from e

    try:
        # Get all the values of the table
        exec1 = select(table1.c[table1_id], table1.c[table1_key])
        exec2 = select(table2.c[table2_id], table2.c[table2_key])

        table1_record = session.execute(select(table1.c[table1_id], table1.c[table1_key])).fetchall()
        table2_record = session.execute(select(table2.c[table2_id], table2.c[table2_key])).fetchall()
    except Exception as e:
        print(e)
        raise Exception("Error fetching records from tables") from e

    try:
        # Create a dictionary with the values of the table
        table1_dict = {table1_key: table1_id for table1_id, table1_key in table1_record}
        table2_dict = {table2_key: table2_id for table2_id, table2_key in table2_record}
    except Exception as e:
        raise Exception("Error creating dictionaries from table records") from e

    try:
        # Create the associations using the dictionaries, with a progress bar
        associations = []
        for index, row in tqdm(dataframe.iterrows(), total=len(dataframe), desc='Creating associations'):
            table1_id_base = table1_dict.get(row[f'{table1_key}'])
            table2_id_base = table2_dict.get(row[f'{table2_key}'])
            # Check that both IDs exist before inserting
            if table1_id_base is not None and table2_id_base is not None:
                associations.append({table1_id: table1_id_base, table2_id: table2_id_base})
                print(f"Association created between {table1_id_base} and {table1_id_base}")
            else:
                print(f"Association NOT created between {table1_id_base} and {table2_id_base}")
    except Exception as e:
        print(e)
        raise Exception("Error creating associations") from e

    try:
        # Save the associations to the CSV file
        associations_df = pd.DataFrame(associations)
        if (table_name is not None):
            associations_df.to_csv(f'new_data/Associations_{table_name}.csv', index=False)
        else:
            associations_df.to_csv('new_data/Associations_{table1}_{table2}.csv'.format(table1=table1, table2=table2), index=False)
    except Exception as e:
        raise Exception("Error saving associations to CSV file", e) from e

    try:
        associations_df = associations_df.drop_duplicates()
        if (table_name is not None):
            associations_df.to_sql(f"{table_name}", con=engine, if_exists='append', index=False)
        else:
            associations_df.to_sql(f"{table1}_{table2}", con=engine, if_exists='append', index=False)
        return True
    except Exception as e:
        raise Exception("Error inserting associations into the database") from e
    
def insert_table_assocation_book(dataframe, table1, table1_key, table1_id):
    """
    Creates and inserts associations between an SQL table and the book table.
    
    Args:
        dataframe (pd.DataFrame): The DataFrame containing data to create associations.
        table1 (str): Name of the first table in the database.
        table1_key (str): Name of the secondary key used to identify records in `table1`.
        table1_id (str): Name of the column containing the ID in `table1`.
    
    Returns:
        bool: True if the associations are successfully inserted.
    
    Raises:
        Exception: If any step in the process fails (connection, data retrieval, etc.).
    """
    attempts = 3
    interval = 3
    for attempt in range(attempts):
        try:
            engine = conexion_db()[0]
            session = conexion_db()[1]
            break
        except Exception as e:
            if attempt < attempts - 1:
                time.sleep(interval)
            else:
                raise Exception("Failed to connect to the database after multiple attempts") from e

    try:
        # Load the tables with the engine
        table1 = Table(table1, MetaData(), autoload_with=engine)
        book = Table('book', MetaData(), autoload_with=engine)
    except Exception as e:
        raise Exception("Error loading tables with the engine") from e

    # la table livre n'as pas de clé secondaire mais on a l'id du livre donc on a pas besoin de la recuperer
    try:
        # Get all the values of the table
        exec1 = select(table1.c[table1_id], table1.c[table1_key])

        table1_record = session.execute(select(table1.c[table1_id], table1.c[table1_key])).fetchall()
    except Exception as e:
        print(e)
        raise Exception("Error fetching records from tables") from e
    
    try:
        # Create a dictionary with the values of the table
        table1_dict = {table1_key: table1_id for table1_id, table1_key in table1_record}
    except Exception as e:
        raise Exception("Error creating dictionaries from table records") from e
    
    try:
        # Create the associations using the dictionaries, with a progress bar
        associations = []
        for index, row in tqdm(dataframe.iterrows(), total=len(dataframe), desc='Creating associations'):
            table1_id_base = table1_dict.get(row[f'{table1_key}'])
            # Check that both IDs exist before inserting
            if table1_id_base is not None:
                associations.append({table1_id: table1_id_base, 'book_id': row['book_id']})
                print(f"Association created between {table1_id_base} and {row['book_id']}")
            else:
                print(f"Association NOT created between {table1_id_base} and {row['book_id']}")
    except Exception as e:
        print(e)
        print(f"coucou{e}")
        raise Exception("Error creating associations") from e
    
    try:
        # Save the associations to the CSV file
        associations_df = pd.DataFrame(associations)
        associations_df.to_csv('new_data/Associations{table1}_book.csv'.format(table1=table1), index=False)
    except Exception as e:
        raise Exception("Error saving associations to CSV file") from e
    
    try:
        associations_df = associations_df.drop_duplicates()
        associations_df.to_sql(f"book_{table1}", con=engine, if_exists='append', index=False)
        return True
    except Exception as e:
        raise Exception("Error inserting associations into the database") from e
    